In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os as os
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import scipy as sp
from sklearn.preprocessing import MinMaxScaler
from scipy.special import boxcox1p
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from imblearn.under_sampling import *
from imblearn.over_sampling import *
from imblearn.combine import *
from sklearn.metrics import log_loss

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
train = pd.read_csv('/content/drive/My Drive/datrain.csv')
test = pd.read_csv('/content/drive/My Drive/datest.csv')
sample_submission = pd.read_csv('/content/drive/My Drive/sample_submission.csv')

sample_submission.describe()

,id,STAR_WHITE_DWARF,STAR_CATY_VAR,STAR_BROWN_DWARF,SERENDIPITY_RED,REDDEN_STD,STAR_BHB,GALAXY,SERENDIPITY_DISTANT,QSO,SKY,STAR_RED_DWARF,ROSAT_D,STAR_PN,SERENDIPITY_FIRST,STAR_CARBON,SPECTROPHOTO_STD,STAR_SUB_DWARF,SERENDIPITY_MANUAL,SERENDIPITY_BLUE
count,10009.000000,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0
mean,204995.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,2889.493756,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,199991.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,202493.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,204995.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,207497.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,209999.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

In [0]:
test_id=test['id']

In [0]:
test = test.drop(columns=['fiberID', 'id'])

In [0]:
train_x = train.drop(columns=['id','type_num', 'fiberID','type'], axis=1)
train_y = train['type_num']
test_x = test
train_x

,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,21.387653,21.572827,22.504317,21.431636,21.478312,21.145409,20.422446,22.749241,21.465534,21.364187,21.020605,21.147340
1,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,20.937731,21.063646,21.360701,20.778968,20.889705,20.639812,20.646660,21.492955,20.758527,20.753925,20.693389,20.512314
2,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,17.095999,17.076449,17.867253,16.738784,16.688874,16.744210,16.808006,17.818063,16.697434,16.641249,16.660177,16.688928
3,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,20.073852,19.794505,20.433907,19.993727,19.985531,19.750917,19.455117,20.770711,20.001699,19.889798,19.758113,19.552855
4,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,19.861718,18.810375,25.859229,22.426929,21.673551,19.610012,18.376141,24.877052,23.147993,21.475342,19.487330,18.375655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199986,20.630780,20.437920,20.050756,20.020974,19.959646,20.948519,20.726709,20.311562,20.323418,20.211336,20.836400,20.536338,20.126561,20.102815,19.796722,20.783295,20.326879,20.014046,20.061940,19.862513
199987,25.283695,22.807243,21.069792,20.187531,19.595598,26.677671,22.818723,20.786120,19.961857,19.457071,28.895442,22.250489,20.091693,18.985791,18.803354,26.921903,21.952440,19.776321,18.990039,18.528306
199988,23.609826,21.902709,20.330747,18.457535,17.326771,24.320816,22.287946,20.711737,18.810560,17.721928,25.982378,22.132993,20.329624,18.507337,17.418521,24.503460,21.897599,20.420524,18.443126,17.331724
199989,21.172003,20.561208,20.031754,19.793644,19.540471,21.704126,20.469748,19.872089,19.603742,19.372909,18.880781,18.065559,17.501307,17.180639,17.175205,19.109086,18.004976,17.440331,17.193743,16.969854


In [0]:
train

,id,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z,type_num
0,0,QSO,601,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,21.387653,21.572827,22.504317,21.431636,21.478312,21.145409,20.422446,22.749241,21.465534,21.364187,21.020605,21.147340,9
1,1,QSO,788,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,20.937731,21.063646,21.360701,20.778968,20.889705,20.639812,20.646660,21.492955,20.758527,20.753925,20.693389,20.512314,9
2,2,QSO,427,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,17.095999,17.076449,17.867253,16.738784,16.688874,16.744210,16.808006,17.818063,16.697434,16.641249,16.660177,16.688928,9
3,3,QSO,864,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,20.073852,19.794505,20.433907,19.993727,19.985531,19.750917,19.455117,20.770711,20.001699,19.889798,19.758113,19.552855,9
4,4,STAR_RED_DWARF,612,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,19.861718,18.810375,25.859229,22.426929,21.673551,19.610012,18.376141,24.877052,23.147993,21.475342,19.487330,18.375655,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199986,199986,QSO,568,20.630780,20.437920,20.050756,20.020974,19.959646,20.948519,20.726709,20.311562,20.323418,20.211336,20.836400,20.536338,20.126561,20.102815,19.796722,20.783295,20.326879,20.014046,20.061940,19.862513,9
199987,199987,GALAXY,362,25.283695,22.807243,21.069792,20.187531,19.595598,26.677671,22.818723,20.786120,19.961857,19.457071,28.895442,22.250489,20.091693,18.985791,18.803354,26.921903,21.952440,19.776321,18.990039,18.528306,7
199988,199988,STAR_RED_DWARF,146,23.609826,21.902709,20.330747,18.457535,17.326771,24.320816,22.287946,20.711737,18.810560,17.721928,25.982378,22.132993,20.329624,18.507337,17.418521,24.503460,21.897599,20.420524,18.443126,17.331724,11
199989,199989,GALAXY,85,21.172003,20.561208,20.031754,19.793644,19.540471,21.704126,20.469748,19.872089,19.603742,19.372909,18.880781,18.065559,17.501307,17.180639,17.175205,19.109086,18.004976,17.440331,17.193743,16.969854,7


In [0]:
# oversample = RandomOverSampler(sampling_strategy='minority')
# ST_train_x, ST_train_y = oversample.fit_resample(train_x, train_y)

In [0]:
# ST=SMOTETomek()

# ST_train_x,ST_train_y = ST.fit_sample(train_x,train_y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

In [0]:
from sklearn.model_selection import train_test_split
ST_X_train, ST_X_valid, ST_y_train, ST_y_valid = train_test_split(train_x, train_y, \
                                                  test_size=0.2, random_state=42, stratify = train_y)

In [0]:
ST_y_train=pd.DataFrame(ST_y_train)
ST_y_train.apply(pd.value_counts)

,type_num
9,39744
7,29877
19,17408
16,11704
5,11694
11,11000
6,10800
14,5705
12,5264
2,5205


In [0]:
ST_X_train=pd.DataFrame(ST_X_train)
ST_X_train.columns=['psfMag_u', 'psfMag_g', 'psfMag_r', 'psfMag_i', 'psfMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i', 'fiberMag_z', 'petroMag_u', 'petroMag_g', 'petroMag_r', 'petroMag_i', 'petroMag_z', 'modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z']
ST_X_train

,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
73778,20.573677,20.429219,20.424495,20.258707,19.991494,20.910386,20.740721,20.699905,20.687257,20.596735,20.745203,20.485740,20.433317,20.345375,20.028567,20.623210,20.468847,20.406305,20.261626,20.094012
161655,21.081430,19.557006,18.716852,18.344772,18.011042,21.268528,19.613222,18.712044,18.259462,17.917436,19.019040,17.647990,16.844239,16.467294,16.192306,19.301172,17.741261,16.914264,16.487286,16.153001
33116,23.483453,22.047574,20.244258,19.755216,19.281269,22.999783,21.809014,20.056598,19.457926,19.006919,23.217947,21.092803,19.247603,18.508602,18.437393,23.477124,20.911872,19.179367,18.580823,18.251391
194910,20.795337,20.624164,20.565313,20.589143,20.247064,21.215549,20.970106,20.911972,20.959909,21.347692,20.695535,20.664013,20.517507,20.794820,20.442538,20.810141,20.653072,20.540206,20.552589,20.272082
151940,23.519520,21.659171,21.515702,21.139677,20.778581,22.043591,21.890512,21.837239,21.560006,21.866900,23.010105,21.745758,21.496951,21.417040,20.973853,22.188588,21.610041,21.458185,21.151328,20.889414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183509,20.176076,20.027881,20.027611,19.756432,20.020500,20.482536,20.382514,20.468871,20.231662,20.194165,20.154691,20.038881,20.090305,19.907859,20.785877,20.101705,20.001037,20.065114,19.799448,19.916447
104904,21.140322,20.807808,20.752861,20.186714,20.142073,21.230802,21.141265,21.022076,20.501877,20.429789,21.506854,20.889375,21.055427,20.330982,20.017713,21.040995,20.874798,20.697512,20.174482,20.151148
136996,19.916782,19.920386,20.105963,20.299882,20.515739,20.182739,20.120187,20.521294,20.638721,21.334979,19.939905,19.757175,20.131013,20.270169,21.258595,19.836183,19.815281,20.115187,20.332078,20.844596
18362,18.131935,17.141955,16.793860,16.724045,16.653979,18.420736,17.451073,17.136876,17.039049,16.994551,18.145806,17.160202,16.848967,16.750696,16.733648,18.127366,17.124334,16.794521,16.711789,16.666436


In [0]:

ST_X_valid=pd.DataFrame(ST_X_valid)
ST_X_valid.columns=[ 'psfMag_u', 'psfMag_g', 'psfMag_r', 'psfMag_i', 'psfMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i', 'fiberMag_z', 'petroMag_u', 'petroMag_g', 'petroMag_r', 'petroMag_i', 'petroMag_z', 'modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z']
ST_X_valid

,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
75310,18.920283,18.087491,17.702514,17.567592,17.577112,19.273233,18.389619,18.066011,17.941260,17.927332,18.960556,18.051156,17.740520,17.625818,17.539976,18.913024,18.031734,17.711605,17.560032,17.553816
143821,20.375570,19.520414,19.049371,18.718658,18.671391,20.114821,19.026575,18.540232,18.165258,18.039742,18.885585,17.758686,17.323396,16.973687,16.937922,18.884392,17.819234,17.343365,16.960823,16.893067
36901,20.366065,18.780415,17.774750,17.384407,17.194126,20.206652,18.502174,17.613084,17.126169,16.773238,18.105746,16.619140,15.814452,15.454999,15.296198,18.400753,16.708202,15.859394,15.418144,15.086678
21113,22.605059,20.461496,19.463100,19.121556,18.878965,22.843298,20.793533,19.764523,19.457499,19.289336,23.487720,20.533015,19.503717,19.170223,18.969542,22.416332,20.482280,19.451416,19.102410,18.907260
46722,19.158927,18.501618,18.157614,18.137338,18.101144,19.772160,18.808347,18.559973,18.602475,18.509624,19.435178,18.473334,18.242095,18.196800,18.230681,19.297938,18.438942,18.185179,18.130410,18.186079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72785,20.760288,19.783735,19.673776,19.745190,19.726940,21.249072,20.089503,20.065184,20.168075,20.422453,21.087144,19.810913,19.782938,19.773742,20.040439,20.961922,19.761458,19.699139,19.764565,19.797933
156636,21.303572,19.857506,18.885045,18.421955,18.047282,21.251831,19.635942,18.647509,18.222394,17.933049,19.008630,17.824116,16.989990,16.590281,16.328151,19.312565,17.885131,17.006889,16.578884,16.291540
125469,19.665636,19.315090,19.031063,18.052946,17.425077,19.895641,19.615312,19.340621,18.427229,17.747420,19.678235,19.288939,19.068805,18.103826,17.448015,19.688090,19.286500,19.013778,18.052920,17.399141
56452,20.344574,20.319208,20.435924,20.596878,20.759164,20.790896,20.702330,20.763942,20.843153,22.718603,20.360078,20.388626,20.562838,20.632982,19.381917,20.378473,20.295923,20.451135,20.547724,20.552636


In [0]:
ST_y_train=pd.DataFrame(ST_y_train)
ST_y_valid=pd.DataFrame(ST_y_valid)

In [0]:
ST_y_valid.apply(pd.value_counts)

,type_num
9,9936
7,7470
19,4352
16,2926
5,2924
11,2750
6,2700
14,1427
12,1316
2,1301


In [0]:
%%time

rnd_clf = RandomForestClassifier(max_depth=20, n_estimators=1000,max_features=10, oob_score=False, n_jobs=-1)
rnd_clf.fit(ST_X_train,ST_y_train)
rnd_clf_probs = rnd_clf.predict_proba(ST_X_valid)
score = log_loss(ST_y_valid, rnd_clf_probs)
print(score)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
rnd_pred = rnd_clf.predict_proba(test)
submission = pd.DataFrame(data=rnd_pred)
submission= pd.concat([test_id,submission],axis=1)
submission.columns=sample_submission.columns
submission.to_csv('/content/drive/My Drive/rnd_clf.csv', index=False)
submission.describe()

,id,STAR_WHITE_DWARF,STAR_CATY_VAR,STAR_BROWN_DWARF,SERENDIPITY_RED,REDDEN_STD,STAR_BHB,GALAXY,SERENDIPITY_DISTANT,QSO,SKY,STAR_RED_DWARF,ROSAT_D,STAR_PN,SERENDIPITY_FIRST,STAR_CARBON,SPECTROPHOTO_STD,STAR_SUB_DWARF,SERENDIPITY_MANUAL,SERENDIPITY_BLUE
count,10009.000000,10009.000000,1.000900e+04,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000
mean,204995.000000,0.011037,3.257692e-02,0.002666,0.012368,0.073868,0.067067,0.186090,0.022818,0.247232,0.000527,0.068829,0.034727,0.000054,0.036824,0.016198,0.073282,0.005457,0.000307,0.108072
std,2889.493756,0.059484,1.692200e-01,0.033111,0.093067,0.242220,0.213975,0.375866,0.091349,0.372580,0.018835,0.245660,0.084080,0.000733,0.128609,0.110339,0.248815,0.065410,0.003288,0.243305
min,199991.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,202493.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000001,0.000000,0.001101,0.000000,0.000000,0.000958,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,204995.000000,0.000003,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000083,0.000051,0.013051,0.000000,0.000000,0.006080,0.000000,0.000155,0.000000,0.000000,0.000000,0.000000,0.000030
75%,207497.000000,0.000818,5.543237e-07,0.000000,0.000000,0.000000,0.004390,0.009669,0.002057,0.365975,0.000000,0.000028,0.027369,0.000000,0.011957,0.000004,0.000000,0.000000,0.000000,0.015047
max,209999.000000,0.870474,9.931640e-01,0.889109,0.995814,0.975295,0.992553,0.999994,0.935039,0.999311,0.989000,0.999603,0.832429,0.030689,0.982562,0.989628,0.998145,0.994348,0.146969,0.999802
